# CrowdFace: Neural-Adaptive Crowd Segmentation with Ad Integration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BlackBoyZeus/CrowdFace/blob/main/CrowdFace_Demo.ipynb)

## Complete Product Demo

This notebook demonstrates the full CrowdFace system, combining:
- **SAM2** (Segment Anything Model 2) for precise crowd detection and segmentation
- **RVM** (Robust Video Matting) for high-quality alpha matte generation
- **BAGEL** (ByteDance Ad Generation and Embedding Library) for intelligent ad placement

### Key Features
- Advanced crowd segmentation with state-of-the-art models
- Robust video matting for realistic ad integration
- Contextual ad placement based on scene understanding
- Multi-platform support with GPU acceleration

In [ ]:
# Install dependencies
!pip install torch torchvision opencv-python transformers diffusers accelerate safetensors huggingface_hub matplotlib tqdm
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install supervision

In [ ]:
# Import libraries
import os
import sys
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import Video, display, HTML
import supervision as sv

# Check PyTorch and CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Clone repositories and download sample data
!git clone https://github.com/PeterL1n/RobustVideoMatting.git

# Add to Python path
sys.path.append('RobustVideoMatting')

# Download a sample video
!wget -O sample_video.mp4 https://pixabay.com/videos/download/video-41758_source.mp4?attachment

# Download RVM weights
!wget -O rvm_mobilenetv3.pth https://github.com/PeterL1n/RobustVideoMatting/releases/download/v1.0.0/rvm_mobilenetv3.pth

In [ ]:
# Set up Hugging Face token for model access
import os

# Try to get token from environment or Colab secrets
HUGGINGFACE_TOKEN = None

try:
    from google.colab import userdata
    if userdata.get('HUGGINGFACE_TOKEN'):
        HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
        print("Using token from Colab secrets")
except:
    if os.environ.get('HUGGINGFACE_TOKEN'):
        HUGGINGFACE_TOKEN = os.environ.get('HUGGINGFACE_TOKEN')
        print("Using token from environment variables")

# If no token is found, prompt the user
if not HUGGINGFACE_TOKEN:
    print("No Hugging Face token found. Please enter your token below.")
    HUGGINGFACE_TOKEN = input("Enter your Hugging Face token: ")

In [ ]:
# Load SAM2 model
from transformers import SamModel, SamProcessor

print("Loading SAM2 model...")
model_id = "facebook/sam2"

try:
    sam_processor = SamProcessor.from_pretrained(model_id, token=HUGGINGFACE_TOKEN)
    sam_model = SamModel.from_pretrained(model_id, token=HUGGINGFACE_TOKEN)
    
    sam_model = sam_model.to(device)
    print("SAM2 model loaded successfully")
except Exception as e:
    print(f"Error loading SAM2 model: {e}")
    print("Will use a placeholder for demonstration purposes")
    sam_model = None
    sam_processor = None

In [ ]:
# Load RVM model
try:
    from model import MattingNetwork
    
    # Load RVM model
    rvm_model = MattingNetwork('mobilenetv3').eval().to(device)
    
    # Load weights
    rvm_model.load_state_dict(torch.load('rvm_mobilenetv3.pth', map_location=device))
    print("RVM model loaded successfully")
except Exception as e:
    print(f"Error loading RVM model: {e}")
    print("Will use a placeholder for demonstration purposes")
    rvm_model = None